In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
import warnings

In [2]:
df = pd.read_csv('StudentsPerformance.csv')
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [3]:
X = df.drop(columns='math score')
print(type(X))
X.head()

<class 'pandas.core.frame.DataFrame'>


,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [4]:
numeric_features = [feature for feature in X.columns if X[feature].dtype != 'O']
categorical_features = [feature for feature in X.columns if X[feature].dtype == 'O']

print(f'Numeric features: {numeric_features}')
print(f'Categorical features: {categorical_features}')

Numeric features: ['reading score', 'writing score']
Categorical features: ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']


In [5]:
num_features = X.select_dtypes(exclude='O').columns
cat_features = X.select_dtypes(include='O').columns

print(f'Numeric features: {num_features}')
print(f'Categorical features: {cat_features}')

Numeric features: Index(['reading score', 'writing score'], dtype='object')
Categorical features: Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course'],
      dtype='object')


In [6]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),
    ]
)

In [7]:
type(X)

pandas.core.frame.DataFrame

In [8]:
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [9]:
X_transformed = preprocessor.fit_transform(X)

In [10]:
X.shape

(1000, 7)

In [11]:
X_transformed.shape

(1000, 19)

In [12]:
from sklearn.model_selection import train_test_split
y = df['math score']
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

In [13]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_squared = r2_score(true, predicted)
    return mae, mse, rmse, r2_squared

In [14]:
models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'AdaBoostRegressor': AdaBoostRegressor()
}

model_performance = {}

for model_name in models.keys():
    model = models[model_name] # create instance
    model.fit(X_train, y_train) # fitting
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test datasets
    train_mae, train_mse, train_rmse, train_r2_squared = evaluate_model(y_train, y_train_pred)
    test_mae, test_mse, test_rmse, test_r2_squared = evaluate_model(y_test, y_test_pred)
    
    model_performance[model_name] = test_r2_squared
    print(model_name)
    print('----------')
    print('Model performance for Training Set')
    print(f'- Root Mean Squared Error: {train_rmse:.4f}')
    print(f'- Mean Squared Error: {train_mse:.4f}')
    print(f'- Mean Absolute Error: {train_mae:.4f}')
    print(f'- R2 Score: {train_r2_squared:.4f}')
    print('----------')
    print('Model performance for Testing Set')
    print(f'- Root Mean Squared Error: {test_rmse:.4f}')
    print(f'- Mean Squared Error: {test_mse:.4f}')
    print(f'- Mean Absolute Error: {test_mae:.4f}')
    print(f'- R2 Score: {test_r2_squared:.4f}')

LinearRegression
----------
Model performance for Training Set
- Root Mean Squared Error: 5.3244
- Mean Squared Error: 28.3487
- Mean Absolute Error: 4.2671
- R2 Score: 0.8743
----------
Model performance for Testing Set
- Root Mean Squared Error: 5.3960
- Mean Squared Error: 29.1167
- Mean Absolute Error: 4.2158
- R2 Score: 0.8803
Lasso
----------
Model performance for Training Set
- Root Mean Squared Error: 6.5938
- Mean Squared Error: 43.4784
- Mean Absolute Error: 5.2063
- R2 Score: 0.8071
----------
Model performance for Testing Set
- Root Mean Squared Error: 6.5197
- Mean Squared Error: 42.5064
- Mean Absolute Error: 5.1579
- R2 Score: 0.8253
Ridge
----------
Model performance for Training Set
- Root Mean Squared Error: 5.3233
- Mean Squared Error: 28.3378
- Mean Absolute Error: 4.2650
- R2 Score: 0.8743
----------
Model performance for Testing Set
- Root Mean Squared Error: 5.3904
- Mean Squared Error: 29.0563
- Mean Absolute Error: 4.2111
- R2 Score: 0.8806
KNeighborsRegressor


In [15]:
model_performance

{'LinearRegression': 0.8803449074540941,
 'Lasso': 0.8253197323627853,
 'Ridge': 0.8805931485028737,
 'KNeighborsRegressor': 0.7838129945787431,
 'RandomForestRegressor': 0.8508584534080277,
 'AdaBoostRegressor': 0.8470134043142405}